### Install and Import Dependencies

In [1]:
# Install Pytorch
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
# Install Transformers, BeautifulSoup4, numpy, pandas
!pip install transformers requests beautifulsoup4 pandas numpy

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

### Instantiate Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


### Encode and Calculate Sentiment

In [28]:
tokens = tokenizer.encode('This was great, but nothing to write home about', return_tensors = 'pt')

In [29]:
result = model(tokens)

result

SequenceClassifierOutput(loss=None, logits=tensor([[-0.8711, -0.3766,  0.5909,  0.6413,  0.0248]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [30]:
result.logits

tensor([[-0.8711, -0.3766,  0.5909,  0.6413,  0.0248]],
       grad_fn=<AddmmBackward>)

In [31]:
int(torch.argmax(result.logits))+1

4

#### Collect Reviews

In [37]:
r = requests.get('https://www.yelp.com/biz/143-street-tacos-sierra-vista')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [38]:
reviews

['This is a gem for sure. A small sort of out of the way taco shop that has the best tacos and burrito snot only in town but probably South Eastern Arizona. \xa0These are real street taco slightly Americanized. The salsas are excellent and green is hot. All are great and you should just eat your way through menu. Staff is very friendly and fast. From order to pick up is in a few minutes. The burritos are huge be forewarned. We go at least once a week.',
 'Such an unmemorable name for a taco joint, yet this was craftily written into our collective memories for a simple food item that truly delivered. Passing through Sierra Vista after a day of exploring, we were in search of tacos and gas, not necessarily in that order. A Yelp search gave us 143 Street Tacos, among other choices. But a street taco sounded like just the ticket. After a circuitous drive, we finally arrived at the somewhat out-of-the-way location. The interior was pleasantly surprising, with a sparkling clean appearance, a

In [41]:
df = pd.DataFrame(np.array(reviews), columns = ['reviews'])

In [42]:
df

,reviews
0,This is a gem for sure. A small sort of out of...
1,"Such an unmemorable name for a taco joint, yet..."
2,We saw that this place was #1 on Yelp for the ...
3,Probably one of the best places to grab lunch ...
4,Forgot to take pics of the tacos because they ...
5,Tried 143 Street Tacos based upon the Yelp rev...
6,Went earlier today. Probably the best authenti...
7,You can tell everything is homemade! Best taco...
8,I had been wanting to try out this place due t...
9,"Aye Dios Mio! Look at this burrito, just look ..."


In [46]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [47]:
sentiment_score(df['reviews'].iloc[1])

4

In [49]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [50]:
df

,reviews,sentiment
0,This is a gem for sure. A small sort of out of...,5
1,"Such an unmemorable name for a taco joint, yet...",4
2,We saw that this place was #1 on Yelp for the ...,5
3,Probably one of the best places to grab lunch ...,5
4,Forgot to take pics of the tacos because they ...,4
5,Tried 143 Street Tacos based upon the Yelp rev...,5
6,Went earlier today. Probably the best authenti...,5
7,You can tell everything is homemade! Best taco...,5
8,I had been wanting to try out this place due t...,4
9,"Aye Dios Mio! Look at this burrito, just look ...",5


In [51]:
df['reviews'].iloc[3]

'Probably one of the best places to grab lunch in Sierra Vista. I have been here several times and never left unhappy. I was also impressed with how quickly the food came out. Authentic Mexican food served fast food style. Highly recommend'

In [52]:
df['sentiment'].mean()

4.7